# 03.03 Basic Models in TensorFlow (Logistic Regression + Data + CE) 

### MNIST DataSet

![Alt_text](./03_images/05.png)

X: 손으로 쓴 숫자 이미지  
Y: 정답 값    
이미지를 보고 숫자를 인지 하는 문제  

### Model 

Inference: Y_predicted = softmax(X * w + b)  
Cross entropy loss: -log(Y_predicted)  # 크로스 엔트로피 사용

In [29]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import numpy as np
import tensorflow as tf
import time
import gzip
import shutil
import struct
import urllib

### 모델에 대한 매개 변수 정의

In [30]:
learning_rate = 0.01
batch_size = 128
n_epochs = 30
n_train = 60000
n_test = 10000

### MNIST 가져오는 Fouction

In [31]:
def safe_mkdir(path):
    """ 디렉토리가 없으면 디렉토리를 만듭니다. """
    try:
        os.mkdir(path)
    except OSError:
        pass

In [32]:
def download_one_file(download_url, 
                    local_dest, 
                    expected_byte=None, 
                    unzip_and_remove=False):
    """ 
    파일이 이미 존재하지 않는 경우. download_url에서 local_dest로 파일 다운로드 한다.
    expected_byte가 제공되며 다운로드 한 파일의 바이트와 같습니다.
    unzip_and_remove가 True이면 파일의 압축을 풀고 zip 파일을 제거합니다.
    """
    if os.path.exists(local_dest) or os.path.exists(local_dest[:-3]):
        print('%s already exists' %local_dest)
    else:
        print('Downloading %s' %download_url)
        local_file, _ = urllib.request.urlretrieve(download_url, local_dest)
        file_stat = os.stat(local_dest)
        if expected_byte:
            if file_stat.st_size == expected_byte:
                print('Successfully downloaded %s' %local_dest)
                if unzip_and_remove:
                    with gzip.open(local_dest, 'rb') as f_in, open(local_dest[:-3],'wb') as f_out:
                        shutil.copyfileobj(f_in, f_out)
                    os.remove(local_dest)
            else:
                print('The downloaded file has unexpected number of bytes')

In [33]:
def download_mnist(path):
    """ 
    데이터 세트가 아직 다운로드되지 않은 경우 다운로드하고 압축 해제합니다.
    다운로드는  http://yann.lecun.com/exdb/mnist 해당 Site 에서 합니다.
    """
    safe_mkdir(path)
    url = 'http://yann.lecun.com/exdb/mnist'
    filenames = ['train-images-idx3-ubyte.gz',
                'train-labels-idx1-ubyte.gz',
                't10k-images-idx3-ubyte.gz',
                't10k-labels-idx1-ubyte.gz']
    expected_bytes = [9912422, 28881, 1648877, 4542]

    for filename, byte in zip(filenames, expected_bytes):
        download_url = os.path.join(url, filename)
        local_dest = os.path.join(path, filename)
        download_one_file(download_url, local_dest, byte, True)

In [34]:
def read_mnist(path, flatten=True, num_train=55000):
    """
     데이터가 경로에 저장되어 있으면 mnist 데이터 세트를 읽습니다.  
     그리고 numpy 배열의 두 튜플을 반환합니다.  
     ((train_imgs, train_labels), (test_imgs, test_labels))  
    """
    imgs, labels = parse_data(path, 'train', flatten)
    indices = np.random.permutation(labels.shape[0]) # 셔플 기능을 제공한다.
    train_idx, val_idx = indices[:num_train], indices[num_train:]
    train_img, train_labels = imgs[train_idx, :], labels[train_idx, :]
    val_img, val_labels = imgs[val_idx, :], labels[val_idx, :]
    test = parse_data(path, 't10k', flatten)
    return (train_img, train_labels), (val_img, val_labels), test # train / test / validation 값들을 넘겨준다.

In [35]:
def parse_data(path, dataset, flatten):
    if dataset != 'train' and dataset != 't10k': # 에러 처리 구문
        raise NameError('dataset must be train or t10k')

    label_file = os.path.join(path, dataset + '-labels-idx1-ubyte') # Label 관련 전체 경로 가져오기
    with open(label_file, 'rb') as file:
        _, num = struct.unpack(">II", file.read(8))
        labels = np.fromfile(file, dtype=np.int8) #int8
        new_labels = np.zeros((num, 10))
        new_labels[np.arange(num), labels] = 1
    
    img_file = os.path.join(path, dataset + '-images-idx3-ubyte')
    with open(img_file, 'rb') as file:
        _, num, rows, cols = struct.unpack(">IIII", file.read(16))
        imgs = np.fromfile(file, dtype=np.uint8).reshape(num, rows, cols) #uint8
        imgs = imgs.astype(np.float32) / 255.0
        if flatten:
            imgs = imgs.reshape([num, -1])

    return imgs, new_labels  # image 와 label을 처리 해서 넘겨 준다.

In [37]:
def show(image):
    """
    픽셀 데이터의 2차원 배열을 생성 합니다. 
    """
    plt.imshow(image, cmap='gray')
    plt.show()

### Step 1: 데이터 읽기

In [52]:
mnist_folder = 'data/mnist'
download_mnist(mnist_folder)
train, val, test = read_mnist(mnist_folder, flatten=True)

data/mnist/train-images-idx3-ubyte.gz already exists
data/mnist/train-labels-idx1-ubyte.gz already exists
data/mnist/t10k-images-idx3-ubyte.gz already exists
data/mnist/t10k-labels-idx1-ubyte.gz already exists


### Step 2: 반복 가능한 데이터 만들기

In [63]:
train_data = tf.data.Dataset.from_tensor_slices(train)
train_data = train_data.shuffle(10000) # 버퍼 사이즈를 주고 데이터를 썩는다.
train_data = train_data.batch(batch_size)

In [72]:
train_data.output_types
train_data.output_shapes # one hot 으로 구성 되어 있어서 10차원이다 .

(TensorShape([Dimension(None), Dimension(784)]),
 TensorShape([Dimension(None), Dimension(10)]))

In [65]:
test_data = tf.data.Dataset.from_tensor_slices(test)
test_data = test_data.batch(batch_size)

In [73]:
test_data.output_types
test_data.output_shapes

(TensorShape([Dimension(None), Dimension(784)]),
 TensorShape([Dimension(None), Dimension(10)]))

In [76]:
iterator = tf.data.Iterator.from_structure(train_data.output_types, train_data.output_shapes)
img, label = iterator.get_next()  # 반복적으로 가져오는 부분

In [87]:
train_init = iterator.make_initializer(train_data) # Train_data 초기화 한다 
test_init = iterator.make_initializer(test_data) # Test_data 초기화 한다   여기 주목하자 global_variables_initializer가 없다.

### Step 3: 가중치 와 편향을 만든다.

In [78]:
# w는 평균이 0이고 표준편차가 0.01인 변수로 초기화 한다. 
# b는 0으로 초기화 한다.
# w는 X 와 Y의 차원에 의존하므로 Y = tf.matmul(X, w) 한다.
# b는 Y에 따라 다르다 .
w = tf.get_variable(name='weights', shape=(784, 10), initializer=tf.random_normal_initializer(0, 0.01))
b = tf.get_variable(name='bias', shape=(1, 10), initializer=tf.zeros_initializer())

### Step 4: 모델을 만든다.

### Model  
Inference: Y_predicted = softmax(X * w + b)  

Cross entropy loss: -log(Y_predicted)  

![Alt_text](./03_images/06.png)

In [80]:
# 로그를 반환하는 모델
# 로그는 나중에 softmax 계층을 통과한다.
logits = tf.matmul(img, w) + b

### Step 5: 손실 함수 정의

In [83]:
# 손실 함수를 softmax _cross_entropy_with_logits로 사용한다. 
entropy = tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=label, name='entropy') #  최신 버젼이 V2라 이것을 사용한다. 기존의 함수는 제거된다고 한다. 
loss = tf.reduce_mean(entropy, name='loss') # 배치에 대한 평균을 계산 한다 . (텐서의 차원에서 요소의 평균을 계산)

### Step 6: 학습 최적화를 설정한다.

In [84]:
# 손실을 최소화하기 위해 0.01의 학습 속도로 기울기 하강을 사용
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

### Step 7: 테스트 세트로 정확도를 계산한다. 

In [85]:
preds = tf.nn.softmax(logits)
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(label, 1)) # 정답을 예측했다면 1을 아니면 0을 반환 한다.
accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32)) # 정확도에 지속적으로 더해 준다.

In [88]:
writer = tf.summary.FileWriter('./graphs/logreg', tf.get_default_graph())
with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())
    
    # 학습 모델 n_epochs 시간
    for i in range(n_epochs):
        sess.run(train_init) # train_data 샘플을 그린다.
        total_loss = 0
        n_batches = 0
        try:
            while True:
                _, l = sess.run([optimizer, loss])
                total_loss += l
                n_batches += 1
        except tf.errors.OutOfRangeError: #  무한 루프를 빠져나오는 부분
            pass
        
        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
    print('Total time: {0} seconds'.format(time.time() - start_time))
    
    # 테스트 모델
    sess.run(test_init) # test_data 샘플을 그린다. 
    total_correct_preds = 0
    try:
        while True:
            accuracy_batch = sess.run(accuracy)
            total_correct_preds += accuracy_batch
    except tf.errors.OutOfRangeError:
        pass
    
    print('Accuracy {0}'.format(total_correct_preds/n_test)) # 왜 10000을 하지
writer.close()

Average loss epoch 0: 0.3676462846786477
Average loss epoch 1: 0.294250045352897
Average loss epoch 2: 0.2849809317914552
Average loss epoch 3: 0.2809635798078637
Average loss epoch 4: 0.27386363545476006
Average loss epoch 5: 0.2701791577907496
Average loss epoch 6: 0.272375334470078
Average loss epoch 7: 0.2680079747078031
Average loss epoch 8: 0.2697126157928345
Average loss epoch 9: 0.2651429588191731
Average loss epoch 10: 0.2663586772978306
Average loss epoch 11: 0.2632614758298841
Average loss epoch 12: 0.2613014935580797
Average loss epoch 13: 0.2587680497834849
Average loss epoch 14: 0.26228235140442846
Average loss epoch 15: 0.2558543503630993
Average loss epoch 16: 0.25939700213629147
Average loss epoch 17: 0.2572868493060733
Average loss epoch 18: 0.2615606948906599
Average loss epoch 19: 0.2586500890899536
Average loss epoch 20: 0.25568016016379347
Average loss epoch 21: 0.256738637526368
Average loss epoch 22: 0.2578590102667032
Average loss epoch 23: 0.25596492590252745
